In [1]:
!pip install -U spacy


  Using cached spacy-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (920 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.9
    Uninstalling spacy-3.0.9:
      Successfully uninstalled spacy-3.0.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.
ru-core-news-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.7.2 which is incompatible.


In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.0.0/ru_core_news_sm-3.0.0.tar.gz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 51.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached spacy-3.0.9-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Using cached thinc-8.0.17-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (659 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.1
    Uninstalling thinc-8.2.1:
      Successfully uninstalled thinc-8.2.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.0.9 which is incompatible.


In [3]:
import pandas as pd

import re

import nltk
from nltk.corpus import stopwords

import spacy

#EDA


In [4]:
# Connecting Google Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
folder_path = "/content/drive/My Drive/hackathon/"

## Исследуем файл dealer

In [6]:
dealer = pd.read_csv(folder_path + "marketing_dealer.csv", sep=';', index_col='id')

In [7]:
display(dealer)

,name
id,
1,Moi_vibor_WB
2,Akson
3,Bafus
5,Castorama
6,Cubatora
7,Komus
9,Megastroy
10,OnlineTrade
11,Petrovich


In [8]:
dealer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 1 to 17
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    18 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes


## Исследуем файл product

In [9]:
product = pd.read_csv(folder_path + "marketing_product.csv", sep=';', index_col='id')

In [10]:
display(product.sample(5))

,Unnamed: 0,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
id,,,,,,,,,,,,,,
91,168,131-5/5,4.680008e+12,Гель эконом-класса для мытья посуды вручную. ...,314.52,682.0,41.0,Гель эконом-класса для мытья посуды вручную. ...,Гель эконом-класса для мытья посуды вручную. ...,Гель эконом-класса для мытья посуды вручную. ...,NaN,NaN,NaN,NaN
343,13,034-09,4.680008e+12,Антисептик лессирующий BiO LASUR / лиственница...,255.00,607.0,31.0,Антисептик лессирующийзащитно-декоративныйPROS...,Антисептик лессирующийзащитно-декоративныйPROS...,Антисептик лессирующийзащитно-декоративныйPROS...,577614089.0,NaN,NaN,NaN
293,481,044-05,4.680008e+12,Антисептик-грунт OSB BASE готовый состав / 5 л,374.00,891.0,20.0,Антисептик-грунт для плит OSB PROSEPT ОSB BASE...,Антисептик-грунт для плит OSB PROSEPT ОSB BASE...,Антисептик-грунт для плит OSB PROSEPT ОSB BASE...,189522749.0,150033476.0,044-05,NaN
394,197,018-1,4.680008e+12,Защита от плесени FUNGI STOP концентрат 1:10 ...,353.00,840.0,20.0,"Защита от плесени PROSEPT FUNGI STOP, концент...","Защита от плесени PROSEPT FUNGI STOP, концент...","Защита от плесени PROSEPT FUNGI STOP, концент...",189522720.0,150033470.0,018-1,NaN
34,494,186-5,4.680008e+12,Средство для мытья полов с полимерным покрытие...,579.74,1354.0,40.0,Профессиональное средство для мытья полов с по...,Профессиональное средство для мытья полов с по...,Профессиональное средство для мытья полов с по...,451107584.0,149705852.0,186-5,NaN


In [11]:
# Удаляем лишние столбцы


columns_to_drop = ["Unnamed: 0", "article", "ean_13", "cost", "category_id", "ozon_article", "wb_article",
                   "ym_article", "wb_article_td", "ozon_name", "name_1c", "wb_name"]

# Удаление столбцов из датафрейма
product = product.drop(columns=columns_to_drop)

In [12]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 245 to 52
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               494 non-null    object 
 1   recommended_price  491 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.6+ KB


## Исследуем файл key

In [13]:
key = pd.read_csv(folder_path + "marketing_productdealerkey.csv", sep=';', index_col='id')

In [14]:
display(key.sample(5))

,key,dealer_id,product_id
id,,,
1809,718000962,17,54
1506,20633314,15,414
1807,970374009,17,286
762,427546,9,55
778,344035,9,81


In [15]:
key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1700 entries, 1 to 2023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   key         1700 non-null   object
 1   dealer_id   1700 non-null   int64 
 2   product_id  1700 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 53.1+ KB


## Исследуем файл price

In [16]:
price = pd.read_csv(folder_path + "marketing_dealerprice.csv", sep=';', index_col='id', parse_dates=['date'])

In [17]:
display(price.sample(5))

,product_key,price,product_url,product_name,date,dealer_id
id,,,,,,
16584,15881593,699.0,https://www.vseinstrumenti.ru/product/universa...,Универсальное моющее и чистящее средство PROSE...,2023-07-26,15
15074,26391151,2335.0,https://vimos.ru/product/grunt-prosept-betonko...,Грунт Бетонконтакт Prosept 12кг,2023-07-25,16
11308,100156073,3960.0,https://www.bafus.ru/100156073/,Просепт 42 антисептик для консервации круглого...,2023-07-21,3
18335,20633002,318.0,https://www.vseinstrumenti.ru/product/sredstvo...,Средство для удаления гипсовой пыли PROSEPT Du...,2023-07-28,15
6932,697564,364.0,https://akson.ru//p/sredstvo_dlya_udaleniya_gi...,Средство для удаления гипсовой пыли концентрат...,2023-07-17,2


In [18]:
price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20416 entries, 2 to 20570
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_key   20416 non-null  object        
 1   price         20416 non-null  float64       
 2   product_url   20182 non-null  object        
 3   product_name  20416 non-null  object        
 4   date          20416 non-null  datetime64[ns]
 5   dealer_id     20416 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.1+ MB


#Объединяем данные


In [19]:
df = price.merge(dealer, left_on='dealer_id', right_on='id')
df = df.merge(key, left_on='product_key', right_on='key')
df = df.merge(product, left_on='product_id', right_on='id')

In [20]:
df.head(5)

,product_key,price,product_url,product_name,date,dealer_id_x,name_x,key,dealer_id_y,product_id,name_y,recommended_price
0,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
1,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-12,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
3,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-13,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0
4,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-14,2,Akson,546227,2,12,Универсальное моющее и чистящее средствоUniver...,222.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17567 entries, 0 to 17566
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_key        17567 non-null  object        
 1   price              17567 non-null  float64       
 2   product_url        17355 non-null  object        
 3   product_name       17567 non-null  object        
 4   date               17567 non-null  datetime64[ns]
 5   dealer_id_x        17567 non-null  int64         
 6   name_x             17567 non-null  object        
 7   key                17567 non-null  object        
 8   dealer_id_y        17567 non-null  int64         
 9   product_id         17567 non-null  int64         
 10  name_y             17567 non-null  object        
 11  recommended_price  17567 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.7+ MB


### Удаляем лишние данные.

In [22]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["product_key", "product_url", "date", "name_x", "key", "dealer_id_x"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

In [23]:
# Переименовываем столбец
df = df.rename(columns={'name_y':'original_name'})

# Выявляем типичные отличия в наименовании товара

In [24]:
# Установка максимальной ширины столбцов для отображения полного текста
pd.set_option('display.max_columns', None)  # Отображение всех столбцов
pd.set_option('display.max_colwidth', None)  # Отображение полного текста в ячейках
df.sample(20)

,price,product_name,dealer_id_y,product_id,original_name,recommended_price
13191,682.0,Просепт Professional Cooky Apple E гель эконом-класса для мытья посуды (5 л),3,92,Гель эконом-класса для мытья посуды вручную. С ароматом яблокаCooky Apple Eконцентрированное средство / 5 л ПЭТ,682.0
11360,1335.0,"PROSEPT 50 - отбеливатель для древесины 1:1, 5л (4)",14,241,Отбеливатель для древесины PROSEPT 50концентрат 1:1 / 5 л,1626.0
2589,570.0,Антисептик для наружных работ PROSEPT ECO EXTERIOR Готовый состав 5л 013-5,15,279,Антисептик ECO EXTERIOR / 5 л,613.0
2670,656.0,Антисептик-грунт для плит OSB PROSEPT ОSB BASE гот. состав 5 л 044-05,15,293,Антисептик-грунт OSB BASE готовый состав / 5 л,891.0
14551,219.0,Средство для удаления ржавчины и минеральных отложений Prosept Bath Extra 750 мл,4,56,"Гелеобразное средство усиленного действия для удаления ржавчины и минеральных отложенийBath Extraконцентрат 1:10-1:100 / 0,75 л",215.0
7503,291.0,"""Кондиционер для белья Prosept Crystal Rinser """"Японский чай"""", бальзам, 2 л""",13,139,Кондиционер для белья с ароматом Японского чаяCrystal Rinserконцентрат / 2 л,319.0
5784,250.0,Bath Acryl,1,63,Средство для чистки акриловых поверхностейBath Acryl концентрат 1:30-1:80 / 1 л,266.0
11,245.0,Чистящее средство Universal Spray спрей универсаль,1,12,"Универсальное моющее и чистящее средствоUniversal Sprayготовый состав / 0,5 л",222.0
1001,129.0,Спрей д/мытья стекол и зеркал PROSEPT Optic Crystal 500мл,9,200,"Средство для мытья стекол и зеркалOptic Crystalготовый состав / 0,5 л",182.0
9780,564.0,Краска резиновая PROSEPT 071-1 цвет зеленый 1 кг,8,424,Краска резиновая зеленый мох Ral 6005 / 1 кг,649.0


# Выявляем типичные отличия в наименовании товара

In [25]:
# Список столбцов, которые нужно удалить
columns_to_drop = ["price", "dealer_id_y", "product_id", "recommended_price"]

# Удаление столбцов из датафрейма
df = df.drop(columns=columns_to_drop)

,product_name,original_name
2052,Спрей д/очистки полков в банях и саунах PROSEPT Universal Wood 500мл,"Спрей для очистки полков в банях и саунах Universal Woodготовый состав / 0,5л"
13956,Кондиционер для белья Crystal Rinser с ароматом Экзотических Цветов 2,Кондиционер для белья с ароматом экзотических цветовCrystal Rinserконцентрат / 2 л
11174,"Отбеливатель для древесины Prosept 50, 1 л",Отбеливатель для древесиныPROSEPT 50концентрат 1:1 / 1 л
9574,Грунт укрепляющий для рыхлых и осыпающихся поверхн. Prosept 3л,Грунт УКРЕПЛЯЮЩИЙготовый состав / 3 л
3039,Спрей д/мытья стекол и зеркал PROSEPT Optic Shine антистат. эффект 500мл,"Средство для мытья стекол и зеркал с антистатическим эффектом Optic Shineконцентрат 1:50-1:100 / 0,5 л"
10252,Резиновая краска PROSEPT SuperRubber (коричневый Ral 8017; 3 кг) 073-3,Краска резиновая коричневый Ral 8017 / 3 кг
2512,Шпатлевка для плит OSB PROSEPT Proplast 7 кг 081-7,Шпатлевка для плит OSBProplast / 7 кг
2189,Просепт Eco Ultra невымываемый антисептик (10 л) коричневый,"Антисептик ECO ULTRA, коричневый / 10 л"
14222,"Средство чистящее д/кухни PROSEPT Cooky Universal 500мл Цитрус, крем","Чистящий крем для кухниCooky Universal с ароматом цитруса готовый состав / 0,5 л"
15977,"Просепт Professional Optic Crystal средство для мытья стекол, зеркал и пластика (1 л)",Средство для мытья стекол и зеркалOptic Crystalготовый состав / 1 л


In [110]:
extra_words = (
    r'(?:готовый\sсостав|для|концентрат|просепт|prosept|средство|невымываемый|гелеобразный|канистра|'
    r'чистящее|спрей|универсальный|универсальная|универсальное|пэт|жидкое|моющее|гель|чистки|'
    r'концентрированное|professional|готовое|superrubber)'
)

def clean_description(text):
    text = text.lower()
    text = re.sub(r'\(\d+\)', '', text)
    text = re.sub(r'(?<=[а-яА-Я])(?=[a-zA-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])', ' ', text)
    text = re.sub(r'\b\d+:\d+\b', '', text)
    text = re.sub(r'\b\d+(?::\d+)?[-:]\d+(?::\d+)?\b', '', text)

    # Замена выражений типа "2 группа" на "2-ая группа"
    text = re.sub(r'(\d+)\sгруппа', r'\1-ая группа', text)

    # Замена объема в литрах и веса в килограммах на миллилитры и граммы
    text = re.sub(r'(\d+[,.]\d+)\s*л', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} мл", text)
    text = re.sub(r'(\d+[,.]\d+)\s*кг', lambda x: f"{int(float(x.group(1).replace(',', '.')) * 1000)} г", text)

    # Добавление пробелов между цифрами и буквами
    text = re.sub(r'(?<=\d)(?=[а-яА-Яa-zA-Z])', ' ', text)
    text = re.sub(r'(?<=[а-яА-Яa-zA-Z])(?=\d)', ' ', text)

    # Удаление выражений вида "007-10бц"
    text = re.sub(r'\b\d+-\d+[а-яА-Яa-zA-Z]*\b', '', text)

    # Удаление слов, соединенных через "/" или точку
    text = re.sub(r'\b\w+[/\.]\w+(?:[/\.]\w+)?\b', '', text)

    # Удаление избыточных слов
    text = re.sub(extra_words, '', text)

    # Удаление запятых, кавычек, дефисов и других символов
    text = re.sub(r'[,"“”/\.()–;+]', '', text)

    # Удаление обособленных пробелами дефисов
    text = re.sub(r'\s-\s', ' ', text)

    # Замена отдельных выражений
    text = re.sub(r'литров|литра|литр', 'л', text)
    text = re.sub(r'\b(мыло-гель|гель-мыло)\b', 'мыло', text)
    text = re.sub(r'osb\s+proplast', 'osbproplast', text)
    text = re.sub(r'\bкор\b', 'коричневый', text)
    text = re.sub(r'eco\s+ultra', 'ecoultra', text)
    text = re.sub(r'гриля\s+и\s+духовых\s+шкафов|удаления\s+пригоревших\s+жиров', 'жироудалитель', text)
    text = re.sub(r'грунт\s+пропиточный', 'грунтовка', text)
    text = re.sub(r'удалитель\s+высолов', 'очиститель фасадов', text)
    text = re.sub(r'с\s+itrus', 'citrus', text)

    return text

# Применяем функцию к столбцам product_name и original_name
df['product_name_normalized'] = df['product_name'].apply(clean_description)
df['original_name_normalized'] = df['original_name'].apply(clean_description)

In [111]:
df.sample(20, random_state=42)

,product_name,original_name,product_name_normalized,original_name_normalized
14843,Спрей антимикробное д/душ.кабин/акриловых ванн PROSEPT Bath Uni 500мл концентрат,"Универсальный концентрат для санитарных комнат с антимикробным эффектомBath Uni готовый состав / 0,5 л",антимикробное акриловых ванн bath uni 500 мл,санитарных комнат с антимикробным эффектом bath uni 500 мл
15011,Средство для удаления цемента Prosept Cement Cleaner 1 л,Удалитель цемента CEMENT CLEANER концентрат 1:2 / 1 л,удаления цемента cement cleaner 1 л,удалитель цемента cement cleaner 1 л
6991,"Жидкое средство для стирки PROSEPT CRYSTAL для стирки спортивной одежды, 3 л","Жидкое моющее средство для стирки спортивной одежды, обуви и пуховиковCrystal Sportконцентрат / 3 л",стирки crystal стирки спортивной одежды 3 л,стирки спортивной одежды обуви и пуховиков crystal sport 3 л
1769,"Шпатлевка для плит OSB PROSEPT Proplast 1,3 кг 081-1","Шпатлевка для плит OSBProplast / 1,3 кг",шпатлевка плит osbproplast 1300 г,шпатлевка плит osbproplast 1300 г
1085,АнтисептикProseptECOULTRAкор5л,Антисептик ECO ULTRA коричневый / 5 л,антисептик ecoultra коричневый 5 л,антисептик ecoultra коричневый 5 л
16322,"Антисептик PROSEPT АНТИПЛЕСЕНЬ, 5 литров, против грибка и плесени (025-5)",Антисептик против грибка и плесени АНТИПЛЕСЕНЬ готовый состав / 5 л,антисептик антиплесень 5 л против грибка и плесени,антисептик против грибка и плесени антиплесень 5 л
3115,"Жидкое мыло PROSEPT Diona Сitrus Е цитрусовый аромат, 5л, ПЭТ",жидкое гель-мыло эконом-класса. C ароматом яблокаDiona Citrus Eготовое средство / 5 л ПЭТ,мыло diona citrus е цитрусовый аромат 5 л,-мыло эконом-класса c ароматом яблока diona citrus e 5 л
3379,Просепт Professional Cooky Grill Gel концентрат гелеобразный для чистки гриля и духовых шкафов (500 мл),"Средство для чистки гриля и духовых шкафовCooky Grill Gelконцентрированное средство 1:1 – 1:50 / 0,5 л",cooky grill gel жироудалитель 500 мл,жироудалитель cooky grill gel 500 мл
8256,Просепт Bio Lasur антисептик лессирующий защитно-декоративный (2.7 л),"Антисептик лессирующий BiO LASUR / белый люкс / 2,7 л",bio lasur антисептик лессирующий защитно-декоративный 2700 мл,антисептик лессирующий bio lasur белый люкс 2700 мл
2830,"Жироудалитель Prosept 0,5 л спрей","Средство для чистки гриля и духовых шкафовCooky Grill концентрированное средство / 0,55 л",жироудалитель 500 мл,жироудалитель cooky grill 550 мл
